In [1]:
from italian_csv_type_prediction.dataframe_generators import SimpleDatasetGenerator
from italian_csv_type_prediction.models import TypePredictor
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [2]:
def dataset_generation(number:int):
    return SimpleDatasetGenerator().build(number, verbose=False)

def _dataset_generation(args):
    return dataset_generation(*args)

def parallel_dataset_generation(number:int):
    processes = min(cpu_count()*5, number)
    with Pool(cpu_count()) as p:
        Xs, ys = list(zip(*tqdm(
            p.imap(_dataset_generation, (
                (number//processes, )
                for _ in range(processes)
            )),
            desc="Creating dataset",
            total=processes,
            leave=False
        )))
    return np.vstack(Xs), np.concatenate(ys)

In [3]:
x_train, y_train = parallel_dataset_generation(2000)
x_test, y_test = parallel_dataset_generation(2000)

In [4]:
model = TypePredictor()

model.fit(x_train, y_train)

In [5]:
y_pred = model._model.predict(x_test)
y_train_pred = model._model.predict(x_train)

In [6]:
accuracy_score(y_test, y_pred), balanced_accuracy_score(y_test, y_pred)

(0.9522287575812859, 0.9456811807128813)

In [7]:
accuracy_score(y_train, y_train_pred), balanced_accuracy_score(y_train, y_train_pred)

(0.9963537227413851, 0.9982410734707551)

In [8]:
X, y = SimpleDatasetGenerator().generate_simple_dataframe()

In [9]:
y

,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,NumericId
0,ItalianFiscalCode,NaN,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,CountryCode,Name,Surname,String,EMail,NaN,NaN,BiologicalSex,NaN,NumericId
1,ItalianFiscalCode,ItalianVAT,CadastreCode,NaN,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,CountryCode,Name,Error,NaN,EMail,PhoneNumber,NaN,BiologicalSex,NaN,NumericId
2,NaN,NaN,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,NaN,Municipality,...,NaN,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,NumericId
3,ItalianFiscalCode,NaN,NaN,Document,NaN,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,NumericId
4,ItalianFiscalCode,ItalianVAT,NaN,Document,NaN,Address,NaN,NaN,Region,NaN,...,NaN,Name,Surname,String,NaN,PhoneNumber,Date,NaN,Error,NumericId
5,ItalianFiscalCode,ItalianVAT,NaN,Document,Error,Address,NaN,ProvinceCode,Region,Municipality,...,CountryCode,Name,NaN,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,NumericId
6,ItalianFiscalCode,ItalianVAT,CadastreCode,Error,NaN,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,CountryCode,Name,Surname,String,EMail,Error,Date,BiologicalSex,Boolean,NumericId
7,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Plate,NaN,ItalianZIPCode,ProvinceCode,Region,NaN,...,CountryCode,NaN,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,NaN
8,ItalianFiscalCode,NaN,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,Error,Name,Surname,String,EMail,PhoneNumber,Date,NaN,Boolean,NaN
9,ItalianFiscalCode,ItalianVAT,NaN,Document,NaN,Address,NaN,ProvinceCode,Region,Municipality,...,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,NumericId


In [10]:
(model.predict_dataframe(X) == y).any()

ItalianFiscalCode    True
ItalianVAT           True
CadastreCode         True
Document             True
Plate                True
Address              True
ItalianZIPCode       True
ProvinceCode         True
Region               True
Municipality         True
Year                 True
Integer              True
Float                True
Country              True
CountryCode          True
Name                 True
Surname              True
String               True
EMail                True
PhoneNumber          True
Date                 True
BiologicalSex        True
Boolean              True
NumericId            True
dtype: bool

In [11]:
from collections import Counter

mask = y_test != y_pred

true_labels = model._embedder._encoder.inverse_transform(y_test[mask])
predicted_labels = model._embedder._encoder.inverse_transform(y_pred[mask])

Counter(zip(true_labels, predicted_labels))

Counter({('Error', 'Country'): 2008,
         ('Surname', 'Error'): 810,
         ('String', 'Error'): 2663,
         ('Error', 'String'): 2093,
         ('Error', 'ItalianFiscalCode'): 67,
         ('Error', 'Surname'): 841,
         ('Country', 'Error'): 2147,
         ('Name', 'Error'): 1101,
         ('Name', 'Surname'): 1530,
         ('Error', 'Document'): 438,
         ('Error', 'Integer'): 328,
         ('Integer', 'Error'): 524,
         ('Country', 'Municipality'): 1509,
         ('CountryCode', 'Boolean'): 561,
         ('Error', 'Boolean'): 209,
         ('Name', 'Region'): 467,
         ('Error', 'Region'): 236,
         ('Error', 'Address'): 371,
         ('String', 'Address'): 2991,
         ('CadastreCode', 'Error'): 509,
         ('Country', 'Region'): 1202,
         ('Name', 'Country'): 4955,
         ('String', 'Municipality'): 228,
         ('Error', 'CountryCode'): 311,
         ('CountryCode', 'NaN'): 152,
         ('Error', 'Name'): 1030,
         ('Surname', 'Na